In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS HERO_WIN;
DROP TABLE IF EXISTS JOGADOR;
DROP TABLE IF EXISTS PARTIDA;
DROP TABLE IF EXISTS PERFORMANCE;
DROP TABLE IF EXISTS HEROI;
DROP TABLE IF EXISTS HEROI_ROLE;

## Criando a tabela de Jogadores

In [3]:
CREATE TABLE JOGADOR(
    PLAYER_ID INTEGER,
    MMR INTEGER,
    RANK INTEGER,
    WIN INTEGER,
    LOSE INTEGER,
    PRIMARY KEY(PLAYER_ID)
) AS SELECT
    PLAYER_ID,
    MMR,
    RANK,
    WIN,
    LOSE
FROM CSVREAD('https://raw.githubusercontent.com/FelipePM01/Dota2Analysis/src_update/final/data/interim/player_data.csv');

## Criando a tabela de Partidas

In [4]:
CREATE TABLE PARTIDA(
    MATCH_ID VARCHAR,
    AVERAGE_MMR INTEGER,
    PLAYER_1_ID INTEGER,
    PLAYER_2_ID INTEGER,
    PLAYER_3_ID INTEGER,
    PLAYER_4_ID INTEGER,
    PLAYER_5_ID INTEGER,
    PLAYER_6_ID INTEGER,
    PLAYER_7_ID INTEGER,
    PLAYER_8_ID INTEGER,
    PLAYER_9_ID INTEGER,
    PLAYER_10_ID INTEGER,
    RADIANT_WIN BOOLEAN,
    FIRST_BLOOD_TIME INTEGER,
    DURATION INTEGER,
    PRIMARY KEY(MATCH_ID)
) AS SELECT
    MATCH_ID,
    AVERAGE_MMR,
    PLAYER_1_ID,
    PLAYER_2_ID,
    PLAYER_3_ID,
    PLAYER_4_ID,
    PLAYER_5_ID,
    PLAYER_6_ID,
    PLAYER_7_ID,
    PLAYER_8_ID,
    PLAYER_9_ID,
    PLAYER_10_ID,
    RADIANT_WIN,
    FIRST_BLOOD_TIME,
    DURATION
FROM CSVREAD('https://raw.githubusercontent.com/FelipePM01/Dota2Analysis/src_update/final/data/processed/match_data.csv');

In [5]:
CREATE TABLE PERFORMANCE(
    MATCH_ID VARCHAR,
    PLAYER_ID INTEGER,
    RADIANT_TEAM BOOLEAN,
    LEVEL INTEGER,
    HERO_ID INTEGER,
    HERO_DAMAGE INTEGER,
    HERO_HEALING INTEGER,
    DAMAGE_TAKEN INTEGER,
    TOWER_DAMAGE INTEGER,
    KILLS INTEGER,
    ASSISTS INTEGER,
    DEATHS INTEGER,
    KDA INTEGER,
    GOLD_PER_MINUTE INTEGER,
    GOLD_SPENT INTEGER,
    EXPERIENCE_PER_MINUTE INTEGER,
    LAST_HITS INTEGER,
    ITEM_0 INTEGER,
    ITEM_1 INTEGER,
    ITEM_2 INTEGER,
    ITEM_3 INTEGER,
    ITEM_4 INTEGER,
    WIN INTEGER,
    PRIMARY KEY(MATCH_ID,PLAYER_ID)
) AS SELECT
    MATCH_ID,
    PLAYER_ID,
    RADIANT_TEAM,
    LEVEL,
    HERO_ID,
    HERO_DAMAGE,
    HERO_HEALING,
    DAMAGE_TAKEN,
    TOWER_DAMAGE,
    KILLS,
    ASSISTS,
    DEATHS,
    KDA,
    GOLD_PER_MINUTE,
    GOLD_SPENT,
    EXPERIENCE_PER_MINUTE,
    LAST_HITS,
    ITEM_0,
    ITEM_1,
    ITEM_2,
    ITEM_3,
    ITEM_4,
    WIN
FROM CSVREAD('https://raw.githubusercontent.com/FelipePM01/Dota2Analysis/src_update/final/data/processed/performance_data.csv');

## Criando a tabela de herois

In [6]:
CREATE TABLE HEROI(
    ID INTEGER,
    NOME VARCHAR,
    ATRIBUTO_PRIMARIO VARCHAR,
    TIPO_ATAQUE VARCHAR,
    PRIMARY KEY(ID,NOME)
) AS SELECT
    ID,
    NOME,
    ATRIBUTO_PRIMARIO,
    TIPO_ATAQUE
FROM CSVREAD('https://raw.githubusercontent.com/FelipePM01/Dota2Analysis/src_update/final/data/interim/hero_data.csv');

In [7]:
CREATE TABLE HEROI_ROLE(
    NOME VARCHAR,
    FUNCAO VARCHAR,
    PRIMARY KEY(NOME,FUNCAO)
) AS SELECT
    NOME,
    FUNCAO
FROM CSVREAD('https://raw.githubusercontent.com/FelipePM01/Dota2Analysis/src_update/final/data/processed/hero_role_data.csv')

## Calculando a taxa de vitoria de um heroi

In [8]:
CREATE VIEW HERO_WIN AS
SELECT HERO_ID,WIN FROM PERFORMANCE


In [9]:
DROP TABLE IF EXISTS HERO_WIN_LOSE;
CREATE VIEW HERO_WIN_LOSE AS
    SELECT HERO_ID,SUM(WIN) AS WINS, COUNT(*) AS MATCHES FROM HERO_WIN
    GROUP BY (HERO_ID)

In [10]:
SELECT * FROM HERO_WIN_LOSE

In [11]:
DROP TABLE IF EXISTS HERO_WINRATE;
CREATE VIEW HERO_WINRATE AS
    
    SELECT HERO_ID,100*WINS/MATCHES AS WINRATE_PORCENTAGEM FROM HERO_WIN_LOSE;

In [12]:
SELECT * FROM HERO_WINRATE

In [13]:
SELECT W.HERO_ID, H.NOME, H.ATRIBUTO_PRIMARIO, H.TIPO_ATAQUE, W.WINRATE_PORCENTAGEM
    FROM HEROI H, HERO_WINRATE W
        WHERE H.ID = W.HERO_ID

## Calculando a taxa de vitoria de um player

In [14]:
DROP TABLE IF EXISTS PLAYER_WINRATE;
CREATE VIEW PLAYER_WINRATE AS
    SELECT PLAYER_ID, MMR,RANK,WIN,LOSE,(WIN+LOSE) AS TOTAL_MATCHES,100*WIN/(WIN+LOSE)AS WINRATE_PORCENTAGEM FROM JOGADOR

In [15]:
SELECT * FROM PLAYER_WINRATE

In [16]:
CREATE TABLE GRAPH(
    MATCH_ID VARCHAR,
    PLAYER_1_ID INTEGER,
    PLAYER_2_ID INTEGER,
)AS SELECT 
    MATCH_ID,
    PLAYER_1_ID,
    PLAYER_2_ID
FROM CSVREAD('https://raw.githubusercontent.com/FelipePM01/Dota2Analysis/src_update/src/data/processed/graph_data.csv')

In [17]:
DROP TABLE IF EXISTS NEW_GRAPH;
CREATE VIEW NEW_GRAPH AS
    SELECT PLAYER_1_ID, PLAYER_2_ID, COUNT(*)AS VEZES FROM GRAPH
    GROUP BY(PLAYER_1_ID,PLAYER_2_ID)
    ORDER BY(VEZES)

In [18]:
SELECT * FROM NEW_GRAPH